# “How likely is a payment to be successful?”

In [2]:
import pandas as pd

# Dataset loading
df = pd.read_csv('truelayer_data_sc_test_data_set.csv')

## Feature Engineering

In [3]:
# Converting and filtering the dataset based on our findings during the EDA
df.status = df.status.str.lower()
filtered_df = df[df.status.isin(['executed','failed'])]
filtered_df.shape

(422715, 20)

In [3]:
filtered_df.dtypes

id                     object
bank_id                object
currency               object
status                 object
api_version            object
failure_reason         object
failure_stage          object
customer_id            object
vertical               object
connectivity_type      object
amount_in_currency    float64
country_id             object
createdat_ts           object
lastupdatedat_ts       object
initiated_at           object
executed_at            object
failed_at              object
authorizing_at         object
authorized_at          object
settled_at             object
dtype: object

We can see that there are some columns that we can assume are only filled once the payment has either completed or failed. If we were to use them in our model training, they would be leaking the target variable into the model, and we wouldn't be replicating real-world performance.

Therefore, we will exclude those columns and the ID column, as it doesn't provide any useful information.

In [4]:
cols_to_exclude = ['id', 'failure_reason', 'failure_stage',
                   'lastupdatedat_ts', 'initiated_at', 'executed_at', 'failed_at', 'authorizing_at',
                   'authorized_at', 'settled_at']

filtered_df = filtered_df.drop(columns=cols_to_exclude)

Using the remaining columns, we will proceed with some feature engineering and creation. We will create examples of possible features that could be useful for the task of predicting how likely a payment is to succeed.

These are just examples of the different possibilities we have, mainly focused on previous behaviours at the point of the payment.

Every new feature will be calculated based on the information available up to the point of that specific payment in order to avoid using future information that would not replicate the status of the payment at the point of prediction in the real world.

In [6]:
# Converting createdat_ts to datetime 
filtered_df.createdat_ts = pd.to_datetime(filtered_df.createdat_ts)

In [7]:
# Sorting dataset by customer_id and creation timestamp so we can calculate values up the the transaction time
filtered_df = filtered_df.sort_values(['customer_id', 'createdat_ts'])

In [8]:
# Number of previous payments - This feature should provide the model with information about if a customer is new or not which can affect probability of success
filtered_df['customer_previous_payment_count'] = filtered_df.groupby('customer_id').cumcount()

In [11]:
# Number of successful payments - Similar to total but it will provide information on how many successful payments the customer had
filtered_df['is_success'] = (filtered_df['status'] == 'executed').astype(int)
filtered_df['customer_success_count'] = filtered_df.groupby('customer_id')['is_success'].cumsum()
filtered_df['customer_success_count'] = filtered_df.groupby('customer_id')['customer_success_count'].shift(1).fillna(0)

# Asserting no leakage from customer to customer
first_rows = filtered_df.groupby('customer_id').first()
assert (first_rows['customer_success_count'] == 0).all(), "Error"

In [10]:
# Number of failed payments - Similar to successful payments but for failure information
filtered_df['is_failure'] = (filtered_df['status'] == 'failed').astype(int)
filtered_df['customer_fail_count'] = filtered_df.groupby('customer_id')['is_failure'].cumsum()
filtered_df['customer_fail_count'] = filtered_df.groupby('customer_id')['customer_fail_count'].shift(1).fillna(0)

# Asserting no leakage from customer to customer
first_rows = filtered_df.groupby('customer_id').first()
assert (first_rows['customer_fail_count'] == 0).all(), "Error"

In [12]:
# Testing that successful + failed payments equal the total number of payments per customer
assert(filtered_df.customer_fail_count + filtered_df.customer_success_count == filtered_df.customer_previous_payment_count).all(), 'Error'

In [13]:
# Payment failure rate - Provides the model information about how many of the previous payments have failed 
filtered_df['fail_rate'] = round(filtered_df.customer_fail_count / filtered_df.customer_previous_payment_count, 4)
filtered_df['fail_rate'] = filtered_df['fail_rate'].fillna(0)

In [14]:
# Example of the new features
filtered_df[filtered_df.customer_id == 'fe85ac10f39a80d1fcae429f389f7e039696ce61'][['status','customer_previous_payment_count','customer_success_count','customer_fail_count', 'fail_rate']]

,status,customer_previous_payment_count,customer_success_count,customer_fail_count,fail_rate
472388,failed,0,0.0,0.0,0.0000
472397,executed,1,0.0,1.0,1.0000
472371,failed,2,1.0,1.0,0.5000
472389,executed,3,1.0,2.0,0.6667
472372,executed,4,2.0,2.0,0.5000


In [14]:
# Time since last transaction - Will provide the model information about payments done in quick succession
filtered_df['time_since_last_transaction'] = filtered_df.groupby('customer_id')['createdat_ts'].diff().dt.total_seconds() / 60

In [15]:
# Time since first transaction - Will provide the model information about for how old the customer is in terms of payments
filtered_df['time_since_first_transaction'] = filtered_df.groupby('customer_id')['createdat_ts'].transform(lambda x: (x - x.iloc[0]).dt.total_seconds() / 60)

As I previously mentioned, there would be a lot more features we could calculate from this handfull of data. Next features could be time wise, for example:
- Payments in the last 7, 30, 120 days
- We could do failure rates for the bank which we haven't used yet

Among many others. 

Model performance is heavily influenced by its features and how much information we can provide for every single payment, but, for now, we will stop here.

In [16]:
filtered_df.to_csv('filtered_df.csv', index=False)